In [9]:
pip install textblob

In [23]:
from textblob import TextBlob
import pandas as pd
data = {
    "steps": [3000, 7000, 10000],
    "sleep_hours": [5, 8, 6],
    "water_liters": [1, 2, 3]
}
df = pd.DataFrame(data)
df
def generate_nudge(steps, sleep_hours, water_liters):
   nudges = []
   if steps < 5000:
       nudges.append("You're below 5000 steps, go for a walk!")
   else:
    nudges.append("Great job hitting your step goal!")
   if sleep_hours < 6:
       nudges.append("You slept less than 6 hours, try resting more tonight.")
   else:
        nudges.append("Nice! You got good sleep.")

   if water_liters < 2:
        nudges.append("Drink more water to stay hydrated.")
   else:
        nudges.append("Hydration on point!")

   return nudges

for i, row in df.iterrows():
  print(f"Day {i+1}: {generate_nudge(row["steps"], row["sleep_hours"], row["water_liters"])}")

def calculate_tokens(steps, sleep_hours, water_liters):
   tokens = 0
   tokens += (steps // 1000) * 5
   tokens += int(water_liters) * 2
   if sleep_hours >= 7:
    tokens += 10
   return tokens
for i, row in df.iterrows():
  print(f"Day {i+1}: Tokens earned = {calculate_tokens(row['steps'], row['sleep_hours'], row['water_liters'])}")

  import json

# --- Create an empty knowledge_base.json if it doesn't exist ---
try:
    with open("knowledge_base.json", "x") as f:
        json.dump([], f)
except FileExistsError:
    pass

# --- Load Knowledge Base ---
def load_kb():
    with open("knowledge_base.json") as f:
        return json.load(f)

# --- Save Knowledge Base ---
def save_kb(kb):
    with open("knowledge_base.json", "w") as f:
        json.dump(kb, f, indent=4)

# --- Add entries to Knowledge Base ---
def add_kb_entry(question, response, intent):
    kb = load_kb()
    new_entry = {"question": question.lower(), "response": response, "intent": intent}
    if new_entry not in kb:
        kb.append(new_entry)
        save_kb(kb)
        print(f"Added: {new_entry}")
    else:
        print(f"Entry already exists: {new_entry}")

# Add the desired entries
add_kb_entry("hello", "Hello! I’m your AI Companion.", "general_info")
add_kb_entry("how much water should I drink", "Drink 2-3 liters daily.", "health_query")
add_kb_entry("sleep tips", "Aim for 7-8 hours of sleep.", "health_query")

# --- Search Knowledge Base ---
def search_kb(user_text):
    kb = load_kb()
    for entry in kb:
        if entry["question"] in user_text.lower():
            return entry["response"], entry["intent"]
    return None, "fallback"

# --- Core Agent Logic ---
def process_input(user_text):
    response, intent = search_kb(user_text)
    if response:
        return {
            "intent": intent,
            "response": response,
            "confidence": 0.87
        }
    else:
        return {
            "intent": "fallback",
            "response": "I’m not sure, can you rephrase?",
            "confidence": 0.50
        }

# --- Quick Test ---
if __name__ == "__main__":
    print(process_input("hello"))
    print(process_input("how much water should I drink"))
    print(process_input("sleep tips"))

def detect_sentiment(user_text):
  polarity = TextBlob(user_text).sentiment.polarity
  if polarity > 0:
    return "positive"
  elif polarity < 0:
    return "negative"
  else:
    return "neutral"

def load_profile():
  with open("user_profile.json") as f:
    return json.load(f)

def process_input(user_text):
  response, intent = search_kb(user_text)
  sentiment = detect_sentiment(user_text)

  if response:
    if intent == "health_query" and "water" in user_text.lower():
        profile = load_profile()
        return {
            "intent": intent,
            "response": f"{response} (Your goal: {profile['hydration_goal']})",
            "confidence": 0.87,
            "sentiment": sentiment
        }
    else:
        return {
            "intent": intent,
            "response": response,
            "confidence": 0.87,
            "sentiment": sentiment
        }
  else:
        return {
            "intent": "fallback",
            "response": "I’m not sure, can you rephrase?",
            "confidence": 0.50,
            "sentiment": sentiment
        }
print(process_input("hello, I feel great today"))
print(process_input("I am so tired and sad"))
print(process_input("water"))


import datetime
def log_interaction(user_text, agent_output):
    log_entry = {
        "timestamp": datetime.datetime.now().isoformat(),
        "user_input": user_text,
        "agent_output": agent_output
    }
    with open("logs.json", "a") as f:
      f.write(json.dumps(log_entry)+ "\n")
print(process_input("how much water should I drink"))
print(process_input("I feel tired"))

Day 1: ["You're below 5000 steps, go for a walk!", 'You slept less than 6 hours, try resting more tonight.', 'Drink more water to stay hydrated.']
Day 2: ['Great job hitting your step goal!', 'Nice! You got good sleep.', 'Hydration on point!']
Day 3: ['Great job hitting your step goal!', 'Nice! You got good sleep.', 'Hydration on point!']
Day 1: Tokens earned = 17
Day 2: Tokens earned = 49
Day 3: Tokens earned = 56
Entry already exists: {'question': 'hello', 'response': 'Hello! I’m your AI Companion.', 'intent': 'general_info'}
Entry already exists: {'question': 'how much water should i drink', 'response': 'Drink 2-3 liters daily.', 'intent': 'health_query'}
Entry already exists: {'question': 'sleep tips', 'response': 'Aim for 7-8 hours of sleep.', 'intent': 'health_query'}
{'intent': 'general_info', 'response': 'Hello! I’m your AI Companion.', 'confidence': 0.87}
{'intent': 'health_query', 'response': 'Drink 2-3 liters daily.', 'confidence': 0.87}
{'intent': 'health_query', 'response'